In [36]:
!pip install mlflow


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
import pandas as pd

In [38]:
stock=pd.read_csv(r"Z:\FAANG project\FAANG_DATA_MLFLOW.csv")

In [39]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
features = [
    'Open', 'High', 'Low', 'Volume',
    'Company_Amazon', 'Company_Apple', 'Company_Facebook', 'Company_Google', 'Company_Netflix'
]
target = 'Close'

X = stock[features]
y = stock[target]
X_log_transformed = X.apply(np.log1p)  # Features
y_log_transformed = np.log1p(y)

X_train, X_test, y_train, y_test = train_test_split(X_log_transformed, y_log_transformed, test_size=0.2, random_state=42)

In [40]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23055 entries, 0 to 23054
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           23055 non-null  object 
 1   Ticker            23055 non-null  object 
 2   Date              23055 non-null  object 
 3   Open              23055 non-null  float64
 4   High              23055 non-null  float64
 5   Low               23055 non-null  float64
 6   Close             23055 non-null  float64
 7   Adj Close         23055 non-null  float64
 8   Volume            23055 non-null  int64  
 9   Year              23055 non-null  int64  
 10  Month             23055 non-null  int64  
 11  Day               23055 non-null  int64  
 12  Day_of_Week       23055 non-null  int64  
 13  Company_Amazon    23055 non-null  float64
 14  Company_Apple     23055 non-null  float64
 15  Company_Facebook  23055 non-null  float64
 16  Company_Google    23055 non-null  float6

In [41]:
#1. Linear_Regression
# Import necessary libraries
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.linear_model import Ridge  # Using Ridge instead of Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_regression

# Set MLflow Tracking URI
mlflow.set_tracking_uri("http://localhost:5000")

# # Set Experiment
mlflow.set_experiment("LinearRegression5")

# Generate synthetic dataset with more noise to prevent overfitting
X, y = make_regression(n_samples=200, n_features=1, noise=30, random_state=42)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling (MinMaxScaler)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Ensure no active MLflow runs
if mlflow.active_run():
    mlflow.end_run()

try:
    with mlflow.start_run(run_name="Linear_Regression_Model"):
        # Set Regularization Parameter (Increased alpha)
        alpha = 1.0  

        # Log Parameters
        mlflow.log_param("model", "Linear__Regression")
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("test_size", 0.2)
        mlflow.log_param("random_state", 42)

        # Model Training (Using Ridge instead of Lasso)
        model = Ridge(alpha=alpha)
        model.fit(X_train, y_train)

        # Predictions
        y_pred = model.predict(X_test)

        # Cross-Validation Score
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring="r2")
        mean_cv_r2 = np.mean(cv_scores)

        # Evaluation Metrics
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        # Log Metrics
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2 Score", r2)
        mlflow.log_metric("Mean CV R2", mean_cv_r2)

        # Log Model
        mlflow.sklearn.log_model(model, "Linear__Regression_model")

        # Print Performance
        print(f"Model Performance:\n MAE: {mae:.4f}\n RMSE: {rmse:.4f}\n R2 Score: {r2:.4f}\n Mean CV R2: {mean_cv_r2:.4f}")

except Exception as e:
    print(f"Error: {e}")
    mlflow.end_run(status="FAILED")

finally:
    # Ensure the run ends even if an error occurs
    if mlflow.active_run():
        mlflow.end_run()


2025/03/04 12:54:40 INFO mlflow.tracking.fluent: Experiment with name 'LinearRegression5' does not exist. Creating a new experiment.
2025/03/04 12:54:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model Performance:
 MAE: 27.0462
 RMSE: 34.7045
 R2 Score: 0.8598
 Mean CV R2: 0.8317
🏃 View run Linear_Regression_Model at: http://localhost:5000/#/experiments/106023816214055466/runs/4d646fb5a247418c9fae22df82c92e27
🧪 View experiment at: http://localhost:5000/#/experiments/106023816214055466


In [42]:
#2. DecisionTree_Regression
#importing necessery libraries
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, cross_val_score
import mlflow
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Ensure no active runs
if mlflow.active_run():
    mlflow.end_run()

try:
    with mlflow.start_run(run_name="DecisionTree_regression_model"):
        # Log Parameters
        mlflow.log_param("model", "Decision Tree Regression")
        mlflow.log_param("test_size", 0.2)
        mlflow.log_param("random_state", 42)
        mlflow.log_param("max_depth", 5)
        mlflow.log_param("min_samples_split", 10)
        mlflow.log_param("min_samples_leaf", 5)
        mlflow.log_param("ccp_alpha", 0.01)

        # Model Training with regularization
        model = DecisionTreeRegressor(
            max_depth=5,
            min_samples_split=10,
            min_samples_leaf=5,
            ccp_alpha=0.01,
            random_state=42
        )

        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')
        mean_cv_r2 = np.mean(scores)
        
        print(f'Cross-Validation R2 Scores: {scores}')
        print(f'Mean Cross-Validation R2 Score: {mean_cv_r2:.2f}')
        
        model.fit(X_train, y_train)

        # Predictions
        y_pred = model.predict(X_test)
        
        # Evaluation Metrics
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        # Log Metrics
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2 Score", r2)
        mlflow.log_metric("Mean CV R2", mean_cv_r2)

        # Log Model
        mlflow.sklearn.log_model(model, "DecisionTree_regression_model")

except Exception as e:
    print(f"Error: {e}")
    mlflow.end_run(status="FAILED")

finally:
    # Ensure the run ends even if an error occurs
    if mlflow.active_run():
        mlflow.end_run()


Cross-Validation R2 Scores: [0.85093091 0.83821302 0.82622612 0.74228458 0.83675616]
Mean Cross-Validation R2 Score: 0.82


2025/03/04 12:54:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run DecisionTree_regression_model at: http://localhost:5000/#/experiments/106023816214055466/runs/11945d7eca5347dcac1ec883cdab26b6
🧪 View experiment at: http://localhost:5000/#/experiments/106023816214055466


In [43]:
#3. RandomForest_Regression

#importing necessery libraries
import mlflow
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Ensure no active runs
if mlflow.active_run():
    mlflow.end_run()

try:
    with mlflow.start_run(run_name="RandomForest_regression_model"):
        # Log Parameters
        mlflow.log_param("model", "Random Forest Regression")
        mlflow.log_param("test_size", 0.2)
        mlflow.log_param("random_state", 42)

        # Define Model & Parameter Grid
        model = RandomForestRegressor(random_state=42)
        param_grid = {
            'n_estimators': [50, 100],
            'max_depth': [5, 10, None],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2, 4]
        }

        # Perform Grid Search with 5-Fold Cross-Validation
        grid_search = GridSearchCV(
            estimator=model, 
            param_grid=param_grid, 
            cv=5, 
            scoring='r2', 
            n_jobs=-1, 
            verbose=2
        )
        grid_search.fit(X_train, y_train)

        # Best parameters from GridSearch
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        print(f"\n🔹 Best Parameters: {best_params}")
        mlflow.log_params(best_params)

        # Print Cross-Validation Results
        cv_results = grid_search.cv_results_
        mean_test_scores = cv_results["mean_test_score"]
        print("\n📊 Cross-Validation R2 Scores:", mean_test_scores)
        print(f"📈 Mean CV R2: {np.mean(mean_test_scores):.4f}")
        mlflow.log_metric("Mean CV R2", np.mean(mean_test_scores))

        # Predictions
        y_pred = best_model.predict(X_test)

        # Evaluation Metrics
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        # Log Metrics
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2 Score", r2)

        # Display Scores
        print("\n📊 Model Performance:")
        print(f"🔸 MAE: {mae:.4f}")
        print(f"🔹 RMSE: {rmse:.4f}")
        print(f"📈 R2 Score: {r2:.4f}")

        # Log Model to MLflow
        mlflow.sklearn.log_model(best_model, "RandomForest_regression_model")

except Exception as e:
    print(f"❌ Error: {e}")
    mlflow.end_run(status="FAILED")

finally:
    # Ensure the run ends even if an error occurs
    if mlflow.active_run():
        mlflow.end_run()


Fitting 5 folds for each of 36 candidates, totalling 180 fits

🔹 Best Parameters: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}

📊 Cross-Validation R2 Scores: [0.81589529 0.81912663 0.81792934 0.82145491 0.81927456 0.82214507
 0.82137402 0.8235414  0.83513238 0.83593198 0.83513238 0.83593198
 0.78579952 0.78951729 0.80406093 0.80707175 0.80628826 0.80903805
 0.81164387 0.81308607 0.83417101 0.83473456 0.83417101 0.83473456
 0.78446011 0.78802634 0.80353145 0.80659925 0.80627528 0.80904398
 0.81162493 0.81309311 0.83417101 0.83473456 0.83417101 0.83473456]
📈 Mean CV R2: 0.8174

📊 Model Performance:
🔸 MAE: 29.5252
🔹 RMSE: 35.6956
📈 R2 Score: 0.8517


2025/03/04 12:55:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run RandomForest_regression_model at: http://localhost:5000/#/experiments/106023816214055466/runs/9c37d7167fca43cfadc1c30f9d960dd9
🧪 View experiment at: http://localhost:5000/#/experiments/106023816214055466


In [44]:
!pip install xgboost



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
#4. XGBoost_regression_model

#importing necessary libraries 
import mlflow
import mlflow.sklearn
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Ensure no active runs
if mlflow.active_run():
    mlflow.end_run()

try:
    with mlflow.start_run(run_name="XGBoost_regression_model"):
        print("Training XGBoost Regression Model")

        # Log Parameters
        mlflow.log_param("model", "XGBoost Regression")
        mlflow.log_param("test_size", 0.2)
        mlflow.log_param("random_state", 42)

        # Initialize Model
        model = XGBRegressor(random_state=42)

        param_grid = {
            'n_estimators': [10, 20],
            'max_depth': [3, 5],
            'learning_rate': [0.01, 0.1]
        }

        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=5,
            scoring='r2',
            n_jobs=-1,
            verbose=2
        )

        # Fit Model with GridSearch
        grid_search.fit(X_train, y_train)

        # Best Parameters
        best_model = grid_search.best_estimator_
        print(f"Best Parameters: {grid_search.best_params_}")
        mlflow.log_params(grid_search.best_params_)

        # Cross-validation R2 Scores
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = cross_val_score(best_model, X_train, y_train, cv=kf, scoring='r2')

        print(f"Cross-Validation R2 Scores: {cv_scores}")
        print(f"Mean CV R2: {np.mean(cv_scores):.4f}")
        mlflow.log_metric("Mean_CV_R2", np.mean(cv_scores))

        # Predictions
        y_pred = best_model.predict(X_test)

        # Evaluation Metrics
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        print("\n📊 Model Performance:")
        print(f"MAE: {mae:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print(f"R2 Score: {r2:.4f}")

        # Log Metrics
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2 Score", r2)

        # Log Model
        mlflow.sklearn.log_model(
            sk_model=best_model,
            artifact_path="XGBoost_regression_model"
        )

except Exception as e:
    print(f"Error: {e}")
    mlflow.end_run(status="FAILED")

finally:
    # Ensure the run ends even if an error occurs
    if mlflow.active_run():
        mlflow.end_run()


Training XGBoost Regression Model
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 20}
Cross-Validation R2 Scores: [0.82141301 0.84798175 0.82680591 0.72997133 0.89099156]
Mean CV R2: 0.8234

📊 Model Performance:
MAE: 32.0083
RMSE: 40.1013
R2 Score: 0.8128


2025/03/04 12:55:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBoost_regression_model at: http://localhost:5000/#/experiments/106023816214055466/runs/94dfbe6e13244f32a14cd7816326e840
🧪 View experiment at: http://localhost:5000/#/experiments/106023816214055466


In [46]:
#5.LightGBM_Regression_Model

#importing necessery libraries
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Ensure no active runs
if mlflow.active_run():
    mlflow.end_run()

try:
    with mlflow.start_run(run_name="LightGBM_Regression_Model"):
        print("Training LightGBM Regression Model")

        # Log Parameters
        mlflow.log_param("model", "LightGBM Regression")
        mlflow.log_param("test_size", 0.2)
        mlflow.log_param("random_state", 42)

        # Initialize Model and GridSearchCV
        model = LGBMRegressor(random_state=42)
        param_grid = {
            'n_estimators': [10, 20],
            'max_depth': [-1, 10],
            'learning_rate': [0.01, 0.1]
        }

        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=5,
            scoring='r2',
            n_jobs=-1,
            verbose=2
        )

        # Fit Model with GridSearch
        grid_search.fit(X_train, y_train)

        # Best Parameters
        best_model = grid_search.best_estimator_
        print(f"Best Parameters: {grid_search.best_params_}")
        mlflow.log_params(grid_search.best_params_)

        # Cross-Validation R² Score
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = cross_val_score(best_model, X_train, y_train, cv=kf, scoring='r2')

        print(f"Cross-Validation R2 Scores: {cv_scores}")
        print(f"Mean CV R2: {np.mean(cv_scores):.4f}")
        mlflow.log_metric("Mean_CV_R2", np.mean(cv_scores))

        # Predictions
        y_pred = best_model.predict(X_test)

        # Evaluation Metrics
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        # Log Metrics
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2 Score", r2)

        # Log Model with Signature
        mlflow.sklearn.log_model(
            sk_model=best_model,
            artifact_path="LightGBM_Regression_Model"
        )

except Exception as e:
    print(f"Error: {e}")
    mlflow.end_run(status="FAILED")

finally:
    # Ensure the run ends even if an error occurs
    if mlflow.active_run():
        mlflow.end_run()


Training LightGBM Regression Model
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000177 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 54
[LightGBM] [Info] Number of data points in the train set: 160, number of used features: 1
[LightGBM] [Info] Start training from score 2.920178
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

z:\FAANG project\env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
z:\FAANG project\env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
z:\FAANG project\env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
z:\FAANG project\env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
z:\FAANG project\env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
z:\FAANG project\env\Lib\site-packages\sklearn\utils\validat

🏃 View run LightGBM_Regression_Model at: http://localhost:5000/#/experiments/106023816214055466/runs/63952bf9874a464d91102abca07a2697
🧪 View experiment at: http://localhost:5000/#/experiments/106023816214055466


In [47]:
#6.XGBoost_serial_regression_model 

# Importing necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Ensure no active runs
if mlflow.active_run():
    mlflow.end_run()

try:
    with mlflow.start_run(run_name="XGBoost_serial_regression_model"):
        print("Training XGBoost Serial Regression Model")

        # Log Parameters
        mlflow.log_param("model", "XGBoost Serial Regression")
        mlflow.log_param("test_size", 0.2)
        mlflow.log_param("random_state", 42)

        # Initialize Model
        model = XGBRegressor(random_state=42)

        param_grid = {
            'n_estimators': [10, 20],
            'max_depth': [3, 5],
            'learning_rate': [0.01, 0.1]
        }

        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=5,
            scoring='r2',
            n_jobs=-1,
            verbose=2
        )

        # Fit Model with GridSearch
        grid_search.fit(X_train, y_train)

        # Best Parameters
        best_model = grid_search.best_estimator_
        print(f"Best Parameters: {grid_search.best_params_}")
        mlflow.log_params(grid_search.best_params_)

        # Cross-validation R2 Scores
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = cross_val_score(best_model, X_train, y_train, cv=kf, scoring='r2')

        print(f"Cross-Validation R2 Scores: {cv_scores}")
        print(f"Mean CV R2: {np.mean(cv_scores):.4f}")
        mlflow.log_metric("Mean_CV_R2", np.mean(cv_scores))

        # Predictions
        y_pred = best_model.predict(X_test)

        # Evaluation Metrics
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        print("\n📊 Model Performance:")
        print(f"MAE: {mae:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print(f"R2 Score: {r2:.4f}")

        # Log Metrics
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2 Score", r2)

        # Log Model
        mlflow.sklearn.log_model(
            sk_model=best_model,
            artifact_path="XGBoost_serial_regression_model"
        )

except Exception as e:
    print(f"Error: {e}")
    mlflow.end_run(status="FAILED")

finally:
    # Ensure the run ends even if an error occurs
    if mlflow.active_run():
        mlflow.end_run()


Training XGBoost Serial Regression Model
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 20}
Cross-Validation R2 Scores: [0.82141301 0.84798175 0.82680591 0.72997133 0.89099156]
Mean CV R2: 0.8234

📊 Model Performance:
MAE: 32.0083
RMSE: 40.1013
R2 Score: 0.8128


2025/03/04 12:55:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBoost_serial_regression_model at: http://localhost:5000/#/experiments/106023816214055466/runs/34b531940a4b438c9266108079da2029
🧪 View experiment at: http://localhost:5000/#/experiments/106023816214055466


In [49]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

registered_models = client.search_registered_models()
print([model.name for model in registered_models])

['LinearRegression']
